</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [1]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [11]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [47]:
d_f = pd.read_csv('./data/Kenya_Connectivity_updated_9_15.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/ken_school_processed.csv',low_memory=False,encoding='utf-8')
d_e = pd.read_csv('./data/Kenya_Ericsson_v2.csv',low_memory=False,encoding='utf-8')
#d_g = pd.read_csv('./data/GSMA_RW_Schools_mobile_coverage_v1.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/kenya_school_geolocation_master_v2.csv',low_memory=False,encoding='utf-8')

In [48]:
d_i.shape

(37903, 27)

In [82]:
d_e.head(2)

,giga_school_id,school_name,LATITUDE,LONGITUDE,uuid,coverage,coverage_type,2G_quality,3G_quality,4G_quality,...,2G_Best_Coverage_provider,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage,3G_coverage,4G_coverage,5G_coverage_present,Distance_to_nearest_tower,Data_Source
0,G_KEN_000026,ST VINCENT OMWARI MIXED,-0.657218,34.651425,3403c6bd-12e8-305a-8c4a-609a5dadcbda,Yes,4G,Fair,Good,Fair,...,Safaricom,Safaricom,Safaricom,None,Yes,Yes,Yes,No,-1.0,Ericsson
1,G_KEN_000057,KANYAWANGA,-0.792565,34.590087,d12ef413-bd69-32b3-bdae-4eaf70928cd1,Yes,4G,Good,Good,Fair,...,Telkom,Safaricom,Safaricom,None,Yes,Yes,Yes,No,-1.0,Ericsson


In [43]:
d_i.columns

Index(['uuid', 'country_co', 'source', 'giga_schoo', 'source_sch',
       'school_nam', 'education_', 'region', 'latitude', 'longitude',
       'fiber_node', 'nearest_sc', 'Schools_wi', 'Schools__1', 'Schools__2',
       'nearest_LT', 'nearest__1', 'nearest_UM', 'nearest__2', 'nearest_GS',
       'nearest__3', '2G', '3G', '4G', 'pop_within', 'pop_with_1',
       'pop_with_2'],
      dtype='object')

In [50]:
d_e.rename(columns={'SCHOOL_CODE':'giga_school_id', 'SCHOOL_NAME':'school_name', '2G_coverage':'2G_quality',
                    '3G_coverage':'3G_quality','4G_coverage':'4G_quality','2G_coverage_present':'2G_coverage', 
                    '3G_coverage_present':'3G_coverage','4G_coverage_present':'4G_coverage',
                    'Coverage':'coverage', 'Coverage_type':'coverage_type'}, inplace=True)
d_i.rename(columns={'giga_schoo': 'giga_school_id','school_nam':'school_name'}, inplace=True)
#d_f.rename(columns={'school_id': 'giga_school_id','name':'school_name'}, inplace=True)

In [56]:
# cross-profiles the sources with master data for completeness and correctness

In [51]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
ec = d_e.copy()
itu = d_i.copy()
#gsma = d_g.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)
print(cross_profile_fb_master.report())
print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())
#print(cross_profile_gsma_master.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0    Master       17  37903
1  Facebook       21  37903

Column Summary
--------------

Number of columns in common: 14
Number of columns in Master but not in Facebook: 3
Number of columns in Facebook but not in Master: 7

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 37,903
Number of rows in Master but not in Facebook: 0
Number of rows in Facebook but not in Master: 0

Number of rows with some compared columns unequal: 34,337
Number of rows with all compared columns equal: 3,566

Column Comparison
-----------------

Number of columns compared with some values unequal: 2
Number of columns compared with all values equal: 12
Total number of values which compare unequal: 34,858

Columns with Unequal Values or Types
------------------------------------

        Column Master

In [58]:
# some colunm names standardization

In [59]:
#d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
#d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)
#d_g.rename(columns={'giga_schoo': 'giga_school_id'}, inplace=True)

In [60]:
# cross-profiles the sources with master data for completeness and correctness

In [61]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [52]:
#d_e['coverage'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g'].isnull()) & (d_f['percent_3g'].isnull())& (d_f['percent_4g'].isnull()), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g']<=0) & (d_f['percent_3g']<=0) & (d_f['percent_4g']<=0), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G']<=0) & (d_i['3G']<=0) & (d_i['4G']<=0), 'NO', 'YES')


In [63]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [53]:
d_f['2G_coverage'] = np.where((d_f['percent_2g'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3g'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4g'] > 0), 'YES', 'NO')

In [54]:
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [66]:
#d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [67]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [55]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [56]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
#d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)


In [57]:
d_e.head(1)

,giga_school_id,school_name,LATITUDE,LONGITUDE,uuid,coverage,coverage_type,2G_quality,3G_quality,4G_quality,...,2G_Best_Coverage_provider,3G_Best_Coverage_provider,4G_Best_Coverage_provider,5G_Best_Coverage_provider,2G_coverage,3G_coverage,4G_coverage,5G_coverage_present,Distance_to_nearest_tower,Data_Source
0,G_KEN_000026,ST VINCENT OMWARI MIXED,-0.657218,34.651425,3403c6bd-12e8-305a-8c4a-609a5dadcbda,Yes,4G,Fair,Good,Fair,...,Safaricom,Safaricom,Safaricom,None,Yes,Yes,Yes,No,-1.0,Ericsson


In [71]:
#d_e.rename(columns={'2G_coverage_present':'2G_coverage', '3G_coverage_present':'3G_coverage','4G_coverage_present':'4G_coverage', 'Coverage':'coverage', 'Coverage_type':'coverage_type'}, inplace=True)

In [72]:
# isolating the needed colunms from the rest

In [58]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [59]:
d_fc = d_f[features]
#d_ec = d_e[features]
d_it = d_i[features]
d_ec = d_e[features]

In [31]:
# more sanity checks for nan on giga_school_id

In [60]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
#d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]
d_ec = d_ec[d_ec['giga_school_id'].notna()]

In [64]:
d_ec.loc[d_ec['2G_coverage'] == 'Yes', '2G_coverage'] = 'YES'
d_ec.loc[d_ec['3G_coverage'] == 'Yes', '3G_coverage'] = 'YES'
d_ec.loc[d_ec['4G_coverage'] == 'Yes', '4G_coverage'] = 'YES'
d_ec.loc[d_ec['coverage'] == 'Yes', 'coverage'] = 'YES'

d_ec.loc[d_ec['2G_coverage'] == 'No', '2G_coverage'] = 'NO'
d_ec.loc[d_ec['3G_coverage'] == 'No', '3G_coverage'] = 'NO'
d_ec.loc[d_ec['4G_coverage'] == 'No', '4G_coverage'] = 'NO'
d_ec.loc[d_ec['coverage'] == 'No', 'coverage'] = 'NO'


In [85]:
d_ec.loc[d_ec['2G_coverage'].isnull(), '2G_coverage'] = 'NO'
d_ec.loc[d_ec['3G_coverage'].isnull(), '3G_coverage'] = 'NO'
d_ec.loc[d_ec['4G_coverage'].isnull(), '4G_coverage'] = 'NO'
d_ec.loc[d_ec['coverage'].isnull(), 'coverage'] = 'NO'
d_ec.loc[d_ec['coverage_type'].isnull(), 'coverage_type'] = 'NO'

In [76]:
# visually comparing the sources for matches and mismatches across the columns

In [62]:
d_it.shape

(37903, 6)

In [88]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_ec.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'Ericsson', 'ITU'])
df_all.head()

Facebook                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_KEN_000396      YES            4G         YES         YES         YES   
G_KEN_000986      YES            4G         YES         YES         YES   
G_KEN_001179      YES            4G         YES         YES         YES   
G_KEN_001228      YES            4G         YES         YES         YES   
G_KEN_001432      YES            4G         YES         YES         YES   

             Ericsson                                                    \
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_KEN_000396      YES            4G         YES         YES         YES   
G_KEN_000986      YES            4G         YES         YES         YES   
G_KEN_001179      YES            4G         YES         YES         YES   
G_KEN_001228      YES            4G         YES         YES         YES   
G_KEN_001432      YES            4G         YES         YES         YES   

                  ITU                                                    
             coverage coverage_type 2G_coverage 3G_coverage 4G_coverage  
G_KEN_000396      YES            4G         YES         YES         YES  
G_KEN_000986      YES            4G         YES         YES         YES  
G_KEN_001179      YES            4G         YES         YES         YES  
G_KEN_001228      YES            4G         YES          NO         YES  
G_KEN_001432      YES            4G         YES          NO         YES

In [89]:
d_ec.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_KEN_000026,YES,4G,YES,YES,YES
1,G_KEN_000057,YES,4G,YES,YES,YES
2,G_KEN_000171,YES,4G,YES,YES,YES
3,G_KEN_000280,YES,4G,YES,YES,YES
4,G_KEN_000299,YES,4G,YES,YES,YES


In [90]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.head()

coverage               coverage_type              2G_coverage  \
             Facebook Ericsson  ITU      Facebook Ericsson ITU    Facebook   
G_KEN_000396      YES      YES  YES            4G       4G  4G         YES   
G_KEN_000986      YES      YES  YES            4G       4G  4G         YES   
G_KEN_001179      YES      YES  YES            4G       4G  4G         YES   
G_KEN_001228      YES      YES  YES            4G       4G  4G         YES   
G_KEN_001432      YES      YES  YES            4G       4G  4G         YES   

                           3G_coverage               4G_coverage                
             Ericsson  ITU    Facebook Ericsson  ITU    Facebook Ericsson  ITU  
G_KEN_000396      YES  YES         YES      YES  YES         YES      YES  YES  
G_KEN_000986      YES  YES         YES      YES  YES         YES      YES  YES  
G_KEN_001179      YES  YES         YES      YES  YES         YES      YES  YES  
G_KEN_001228      YES  YES         YES      YES   NO         YES      YES  YES  
G_KEN_001432      YES  YES         YES      YES   NO         YES      YES  YES

In [91]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [92]:
# coverage comparison report

In [93]:
# create a separate dfs
fbc = d_fc.copy()
#ecc = d_ec.copy()
ituc = d_it.copy()
ecc = d_ec.copy()

# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)

cross_profile_facebook_ec = datacompy.Compare(
fbc,
ecc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='GSMA'
)


#print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
#print(cross_profile_Ericsson_itu.report())
print(cross_profile_facebook_ec.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0  Facebook        6  37903
1       ITU        6  37903

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in ITU: 0
Number of columns in ITU but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 37,903
Number of rows in Facebook but not in ITU: 0
Number of rows in ITU but not in Facebook: 0

Number of rows with some compared columns unequal: 13,575
Number of rows with all compared columns equal: 24,328

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 21,663

Columns with Unequal Values or Types
------------------------------------

          Column Facebook dtype IT

In [39]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [98]:
import statistics as stats
df_c = pd.concat([d_fc,d_ec,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])
coverage['coverage_type'] = coverage.apply((lambda x: coverage_type(x)),axis=1)
#coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])


In [99]:
coverage.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_KEN_000001,YES,4G,YES,YES,YES
1,G_KEN_000002,YES,4G,YES,YES,YES
2,G_KEN_000003,YES,4G,YES,YES,YES
3,G_KEN_000004,YES,4G,YES,YES,YES
4,G_KEN_000005,YES,4G,YES,YES,YES


In [101]:
coverage['coverage'] = np.where((coverage['2G_coverage']=='YES') | (coverage['3G_coverage']=='YES') | (coverage['4G_coverage']=='YES'), 'YES', 'NO')

In [102]:
coverage[coverage['coverage']=='NO'].head(10)

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
79,G_KEN_000080,NO,None,NO,NO,NO
886,G_KEN_000887,NO,None,NO,NO,NO
892,G_KEN_000893,NO,None,NO,NO,NO
893,G_KEN_000894,NO,None,NO,NO,NO
900,G_KEN_000901,NO,None,NO,NO,NO
904,G_KEN_000905,NO,None,NO,NO,NO
908,G_KEN_000909,NO,None,NO,NO,NO
917,G_KEN_000918,NO,None,NO,NO,NO
926,G_KEN_000927,NO,None,NO,NO,NO
964,G_KEN_000965,NO,None,NO,NO,NO


In [97]:
#coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

In [103]:
master.head()

,index,uuid,country,country_code,source,giga_school_id,source_school_id,school_name,education_level,latitude,longitude,hex8,dq_score,real_id,source_timestamp,master_timestamp,school_archive
0,1792,edac19dd-0403-3260-a8a5-58e490d819c9,Kenya,KEN,"Open Data Kenya, School 2007",G_KEN_000001,200731995.0,NAMAWANGA GIRLS SEC,Secondary,0.682084,34.700794,887a4d503bfffff,0.8,NaN,2007,15:59.0,No
1,1806,de245b16-5256-3e1e-b323-8299c3f91254,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000002,25159.0,OLEPOLOS,Primary,-0.502607,35.810312,887a6a0451fffff,0.8,NaN,2016,15:59.0,No
2,1822,9b4cc200-3397-3cc3-ac13-7abc20f853eb,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000003,22972.0,OLE POLOS,Primary,-0.502607,35.810312,887a6a0451fffff,0.8,NaN,2016,15:59.0,No
3,1829,904261cb-5a2b-35e3-bace-8bbfacfaa838,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000004,14448.0,KWA NZILI DEB,Primary,-0.904374,38.173953,887a448e97fffff,0.8,NaN,2016,15:59.0,No
4,1843,65fc5374-0586-31a9-8864-449df0d7ee4b,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000005,45504.0,ESOIT EMPURPUTIA PRIMARY,Primary,-1.089263,35.009815,887a6828b5fffff,0.8,NaN,2016,15:59.0,No


In [104]:
master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/KE_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
#master_w_coverage_connectivity['connectivity_availability'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'connectivity'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'coverage_type'] = 'None'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage_type'] = 'no coverage'
master_w_coverage_connectivity.to_csv('./data/KE_master_w_coverage_connectivity.csv', encoding='utf-8')

In [105]:
master_w_coverage.head(2)

,index,uuid,country,country_code,source,giga_school_id,source_school_id,school_name,education_level,latitude,...,dq_score,real_id,source_timestamp,master_timestamp,school_archive,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,1792,edac19dd-0403-3260-a8a5-58e490d819c9,Kenya,KEN,"Open Data Kenya, School 2007",G_KEN_000001,200731995.0,NAMAWANGA GIRLS SEC,Secondary,0.682084,...,0.8,NaN,2007,15:59.0,No,YES,4G,YES,YES,YES
1,1806,de245b16-5256-3e1e-b323-8299c3f91254,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000002,25159.0,OLEPOLOS,Primary,-0.502607,...,0.8,NaN,2016,15:59.0,No,YES,4G,YES,YES,YES


In [106]:
master_w_coverage_connectivity[master_w_coverage_connectivity['coverage_type']=='no coverage'].head(10)

,index,uuid,country,country_code,source,giga_school_id,source_school_id,school_name,education_level,latitude,...,real_id,source_timestamp,master_timestamp,school_archive,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
79,4812,5d1f073d-df85-3441-bf55-12edeb35cea4,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000080,44175.0,SALALMA,Primary,0.923000,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
886,9879,9b592610-1f6e-31a6-88d5-4af87c327e63,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000887,27715.0,LATAKWENY,Primary,1.547281,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
892,9886,0c789608-fd62-35d8-ad96-9cd9da529985,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000893,27718.0,LOIKUMKUM,Primary,1.649145,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
893,9887,71494fc5-50ac-3388-86f6-89740f770cd8,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000894,27719.0,LOODUA,Primary,1.723560,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
900,9894,46631aa7-c6f2-3f85-8a9e-580a6ae1cf32,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000901,27698.0,SWARI,Primary,1.103571,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
904,9898,4547c546-55c4-3618-8032-5f422741b13e,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000905,27673.0,LKISIN,Primary,1.016558,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
908,9902,d0876b86-e667-3826-969e-a5a5cf3a5f45,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000909,27675.0,LOLKUNIYANI,Primary,1.067325,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
917,9911,774981d3-187e-3470-b3a3-009828ff41ae,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000918,27684.0,NDONYO NASIPA,Primary,1.330304,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
926,9922,1efbd4f5-f843-3bc7-9b48-44f10e28ba42,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000927,27690.0,OROMODEI,Primary,1.272993,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO
964,9964,fec8c350-0c4d-3825-8562-c8d1d2d0a1dd,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000965,27729.0,SEREN,Primary,1.871840,...,NaN,2016,15:59.0,No,NO,no coverage,NO,NO,NO,NO


In [107]:
master_w_coverage_connectivity.head()

,index,uuid,country,country_code,source,giga_school_id,source_school_id,school_name,education_level,latitude,...,real_id,source_timestamp,master_timestamp,school_archive,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity
0,1792,edac19dd-0403-3260-a8a5-58e490d819c9,Kenya,KEN,"Open Data Kenya, School 2007",G_KEN_000001,200731995.0,NAMAWANGA GIRLS SEC,Secondary,0.682084,...,NaN,2007,15:59.0,No,YES,4G,YES,YES,YES,NaN
1,1806,de245b16-5256-3e1e-b323-8299c3f91254,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000002,25159.0,OLEPOLOS,Primary,-0.502607,...,NaN,2016,15:59.0,No,YES,4G,YES,YES,YES,NaN
2,1822,9b4cc200-3397-3cc3-ac13-7abc20f853eb,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000003,22972.0,OLE POLOS,Primary,-0.502607,...,NaN,2016,15:59.0,No,YES,4G,YES,YES,YES,NaN
3,1829,904261cb-5a2b-35e3-bace-8bbfacfaa838,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000004,14448.0,KWA NZILI DEB,Primary,-0.904374,...,NaN,2016,15:59.0,No,YES,4G,YES,YES,YES,NaN
4,1843,65fc5374-0586-31a9-8864-449df0d7ee4b,Kenya,KEN,"Ministry of Education, 2016",G_KEN_000005,45504.0,ESOIT EMPURPUTIA PRIMARY,Primary,-1.089263,...,NaN,2016,15:59.0,No,YES,4G,YES,YES,YES,NaN


In [108]:
master_w_coverage_connectivity['coverage'].value_counts()

YES    37783
NO       120
Name: coverage, dtype: int64